# Quantifying delays between exposures.
Craig Lage - 07-Jan-26

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors, plotMountErrors, \
        getAltAzOverPeriod, getAzElRotHexDataForExposure
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from lsst.summit.extras.slewTimingSimonyi import inPositionTopics, getDomeData

In [ ]:
client = makeEfdClient()
butler = butlerUtils.makeDefaultButler("LSSTCam", embargo=True)

In [ ]:
expId = 2026010700034
dataId1 = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord1 = getExpRecordFromDataId(butler, dataId1)
dataId2 = {'exposure':expId + 1, 'instrument':'LSSTCam'}
expRecord2 = getExpRecordFromDataId(butler, dataId2)
shutterClose = expRecord1.timespan.end.utc
shutterOpen = expRecord2.timespan.begin.utc


In [ ]:
inPositionTopics

In [ ]:
topic = inPositionTopics['Rotator']
inPosition = getEfdData(client, topic, begin=shutterClose, end=shutterOpen)

In [ ]:
inPosition

In [ ]:
getAzElRotHexDataForExposure?

In [ ]:
mountData = getAzElRotHexDataForExposure(client, expRecord=expRecord1, postPadding=10.0)

In [ ]:
mountData.rotationData['demandPosition'].plot()
mountData.rotationData['actualPosition'].plot()

In [ ]:
rotationData = getEfdData(
    client,
    "lsst.sal.MTRotator.rotation",
    begin=shutterClose,
    end=shutterOpen,
    postPadding=4.0
)

diff = rotationData['demandPosition'] - rotationData['actualPosition']

In [ ]:
fig, axs = plt.subplots(2,1)
plt.subplots_adjust(hspace=0.5)
plt.suptitle(f"Rotator inPosition {expId}")
rotationData['demandPosition'].plot(ax = axs[0], color='red', label='Demand')
rotationData['actualPosition'].plot(ax = axs[0], ls='--', color='blue', label='actual')
axs[0].axvline(Time(inPosition.index[0]).isot, ls='--', color='red', label="InPos False")
axs[0].axvline(Time(inPosition.index[1]).isot, ls='--', color='green', label="InPos True")
axs[0].axvline(Time(inPosition.index[2]).isot, ls='--', color='red')
axs[0].axvline(Time(inPosition.index[3]).isot, ls='--', color='green')
axs[0].legend()
diff.plot(ax = axs[1], label = 'Difference')
axs[1].legend()
fig.savefig(f"/home/cslage/DATA/Rotator_inPosition_{expId}.png")

In [ ]:
plt.plot(rotationData['actualPosition'].values - rotationData['demandPosition'].values)